In [2]:
import pandas as pd
from tensorflow.keras.layers import LSTM, Embedding, Input, Dense
from tensorflow.keras.models import Sequential, Model
from keras.preprocessing.sequence import pad_sequences
from nltk.tokenize import word_tokenize
from sklearn.preprocessing import LabelBinarizer
import numpy as np

In [165]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [4]:
import nltk

In [5]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [48]:
def load_data():
    test = pd.read_table("/content/drive/MyDrive/nlp/lab2/test_en.txt")
    train = pd.read_table("/content/drive/MyDrive/nlp/lab2/train_en.txt")
    val = pd.read_table("/content/drive/MyDrive/nlp/lab2/val_en.txt")
    return test, train, val

In [49]:
def tokenize(data):
    data['Sentence_Tokens'] = data['Sentence'].apply(lambda x: word_tokenize(x.lower()))


In [50]:
def create_vocabulary(sentences):
    vocab = set()
    for sentence in sentences:
        vocab.update(sentence)
    vocab = list(vocab)

    w_to_i = {word: index for index, word in enumerate(vocab)}

    return len(vocab), w_to_i

In [126]:
test, train, val = load_data()

In [127]:
test

,Sentence,Label
0,So maybe you should be more retarded.,1
1,THERES A MEGATHREAD FOR VACCINE OR COVID RELAT...,1
2,the fuck.... you are on drugs,1
3,NTA. Dump his ass. Tablemanners are no rocke...,1
4,Youre soft as baby shit,1
...,...,...
393,"NTA, your brother is not thinking straight. Ho...",0
394,Formula one…nascar….it exactly the same except...,0
395,"BRB, gonna go call the mods.",0
396,Carlson is the problem,0


In [128]:
test = test.append(val)

<ipython-input-128-e36f5e548249>:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  test = test.append(val)


In [129]:
test

,Sentence,Label
0,So maybe you should be more retarded.,1
1,THERES A MEGATHREAD FOR VACCINE OR COVID RELAT...,1
2,the fuck.... you are on drugs,1
3,NTA. Dump his ass. Tablemanners are no rocke...,1
4,Youre soft as baby shit,1
...,...,...
391,Hunting for karma right here,0
392,Anyone who is still huffing paint thinner by c...,0
393,And you have never made a wrong decision?,0
394,You couldn't maybe just come up with the answe...,0


In [130]:
tokenize(train)
tokenize(test)

In [131]:
train.head()

,Sentence,Label,Sentence_Tokens
0,Pussy nobody asked for your input.,1,"[pussy, nobody, asked, for, your, input, .]"
1,"Ok, this makes no sense. This will create vigi...",1,"[ok, ,, this, makes, no, sense, ., this, will,..."
2,so fucking true. the amount of up and coming r...,1,"[so, fucking, true, ., the, amount, of, up, an..."
3,Go f yourself Republican scum who put us here ...,1,"[go, f, yourself, republican, scum, who, put, ..."
4,Dumb fucking take. People want to do things.,1,"[dumb, fucking, take, ., people, want, to, do,..."


In [132]:
x_train = train['Sentence_Tokens']
y_train = train['Label']
x_test = test['Sentence_Tokens']
y_test = test['Label']

In [133]:
print(x_train.shape, x_test.shape, y_train.shape, y_test.shape)

(3168,) (794,) (3168,) (794,)


In [134]:
vocab_len_train, w_to_i_train = create_vocabulary(x_train.values.tolist())
vocab_len_test, w_to_i_test = create_vocabulary(x_test.values.tolist())

In [135]:
vocab_len_train

4276

In [136]:
vocab_len_test

1854

In [137]:
def map_to_index(data, w_to_i):
    data_to_index = data.apply(lambda x: [int(w_to_i[word]) for word in x])
    print(data_to_index.info())
    return data_to_index

In [138]:
x_train = map_to_index(x_train, w_to_i_train)
x_test = map_to_index(x_test, w_to_i_test)

<class 'pandas.core.series.Series'>
RangeIndex: 3168 entries, 0 to 3167
Series name: Sentence_Tokens
Non-Null Count  Dtype 
--------------  ----- 
3168 non-null   object
dtypes: object(1)
memory usage: 24.9+ KB
None
<class 'pandas.core.series.Series'>
Int64Index: 794 entries, 0 to 395
Series name: Sentence_Tokens
Non-Null Count  Dtype 
--------------  ----- 
794 non-null    object
dtypes: object(1)
memory usage: 12.4+ KB
None


Average length of a sentence

In [151]:
sum = 0
for i in x_train:
  sum += len(i)
print(sum/len(x_train))

13.990214646464647


In [152]:
x_train = pad_sequences(x_train, 20)
x_test = pad_sequences(x_test, 20)

Препорачано е да се користи embedding_size помеѓу 50 и 300. Но бидејќи нашето множество е мало, за да се избегне overfitting се користи што помала вредност, но сепак не е многу мала за да не ги забележи семантичките врски.

Исто така се препорачува ниска вредност за LSTM units параметарот кога станува збор за мало множество како нашето со цел да се избегне overfitting.

In [157]:
embedding_size = 100
model = Sequential()
model.add(Embedding(input_dim=vocab_len_train, output_dim=embedding_size, input_length=x_train.shape[1]))
model.add(LSTM(units=32))
model.add(Dense(units=1, activation='sigmoid'))
# Compile the model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [158]:
print(model.summary())

Model: "sequential_11"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_11 (Embedding)    (None, 20, 100)           427600    
                                                                 
 lstm_13 (LSTM)              (None, 32)                17024     
                                                                 
 dense_9 (Dense)             (None, 1)                 33        
                                                                 
Total params: 444657 (1.70 MB)
Trainable params: 444657 (1.70 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None


Параметрите batch_size и epochs укажуваат на моделот како да учи. Првично беа поставени на 32 и 3 соодветно.

Ја поставив epochs=10, бидејќи е препорачано вредностите на epochs да бидат меѓу 1 и 10. Но после испробување воочив дека подобри резултати добивам кога epochs се поставени на повисоки вредности. Бидејќи го поставувам batch_size на најниска можна вредност со цел да има можност моделот да генерализира. (Се поради тоа што множеството е многу мало за да се добијат добри резултати)

In [164]:
# Train the model
batch_size = 16
epochs = 20
model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs)

prediction = model.predict(x_test)
y_pred = prediction.round().astype(int)

accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)

Epoch 1/20
198/198 [==============================] - 4s 19ms/step - loss: 0.0092 - accuracy: 0.9921
Epoch 2/20
198/198 [==============================] - 4s 19ms/step - loss: 0.0091 - accuracy: 0.9931
Epoch 3/20
198/198 [==============================] - 5s 28ms/step - loss: 0.0100 - accuracy: 0.9905
Epoch 4/20
198/198 [==============================] - 3s 17ms/step - loss: 0.0552 - accuracy: 0.9782
Epoch 5/20
198/198 [==============================] - 3s 17ms/step - loss: 0.0276 - accuracy: 0.9871
Epoch 6/20
198/198 [==============================] - 4s 22ms/step - loss: 0.0262 - accuracy: 0.9877
Epoch 7/20
198/198 [==============================] - 4s 20ms/step - loss: 0.0110 - accuracy: 0.9915
Epoch 8/20
198/198 [==============================] - 3s 17ms/step - loss: 0.0095 - accuracy: 0.9924
Epoch 9/20
198/198 [==============================] - 3s 17ms/step - loss: 0.0093 - accuracy: 0.9918
Epoch 10/20
198/198 [==============================] - 5s 24ms/step - loss: 0.0092 - accura